# ARI with sweeping dimension reduction components and Louvain resolution

In [1]:
import sys, os
import glob
import shutil

import pandas as pd
import numpy as np

from sklearn.metrics import (silhouette_score, silhouette_samples,   
                             adjusted_rand_score, adjusted_mutual_info_score,   
                             normalized_mutual_info_score)
pd.set_option('display.max_colwidth', None)

In [7]:
pred_label_root = './'
true_label_root = '../../../../Dataset/scMINER_13datasets_h5ad_input_true_label/'

In [8]:
def eval_pred_label(true_label_df,pred_files,component):
    eval_df = []
    for each_file in pred_files:
        louvain_res = each_file.split('_')[-1]
        louvain_res = float(louvain_res.split('.txt')[0])
        #louvain_res = np.round(np.log(louvain_res),1)
        
        pred_df = pd.read_table(each_file)
        
        merge_df = pred_df.merge(true_label_df,left_on = 'ID',right_on='cell', how='inner') 
        #print(merge_df)
        #break
    
        pred_k = len(set(merge_df['label']))
        #print(pred_k)
        #break
        ARI = np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

        eval_df.append([component,louvain_res,pred_k,ARI])
    
    return eval_df

In [9]:
for folder in glob.glob(pred_label_root+'*/'):
    dataset = folder.split('/')[1]
    #print(dataset)

    # true label
    true_label_file = glob.glob(true_label_root + dataset+ '*true_label.txt')[0]
    #print(true_label_file)
    true_label_df = pd.read_table(true_label_file)

    summary_df = []
    for folder in glob.glob(pred_label_root + dataset + '/'+ dataset + '*/'):
        #print(folder)
        #break
        component = int(folder.split('_')[-1][:-1])
        #print(component)
        #break
        pred_files = glob.glob(folder + 'clustering*.txt')
        #print(pred_files)
        #break
        eval_df = eval_pred_label(true_label_df,pred_files,component)
        #eval_df = pd.DataFrame(eval_df)
        #print(eval_df)
        
        summary_df.extend(eval_df)
    
    summary_df = pd.DataFrame(summary_df)
    summary_df.rename(columns = {0:'component',1:'louvain_res',2:'pred_k',3:'ARI'},inplace=True)
    summary_df.to_csv(dataset + '_scMINER_sweep_component_summary.csv',sep='\t')

    #break